In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [7]:
def get_stock_prices(tickers, start_date, end_date):
    prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
    return prices.dropna()

tickers = ["NVDA", "LLY", "GOOG", "JPM", "^GSPC"]
start_date = "2020-01-01"
end_date = "2025-12-16"

prices = get_stock_prices(tickers, start_date, end_date)
prices

[*********************100%***********************]  5 of 5 completed


Ticker,GOOG,JPM,LLY,NVDA,^GSPC
Date,,,,,
2020-01-02,67.859543,119.573341,122.998077,5.971077,3257.850098
2020-01-03,67.526535,117.995438,122.588722,5.875505,3234.850098
2020-01-06,69.191551,117.901634,123.044563,5.900145,3246.280029
2020-01-07,69.148384,115.897232,123.277153,5.971575,3237.179932
2020-01-08,69.693291,116.801323,124.393547,5.982776,3253.050049
...,...,...,...,...,...
2025-12-09,317.750000,300.510010,982.219971,184.970001,6840.509766
2025-12-10,321.000000,310.109985,993.640015,183.779999,6886.680176
2025-12-11,313.700012,317.380005,1009.380005,180.929993,6901.000000


In [8]:
def calculate_returns(prices, return_type='simple'):
    prices = prices.sort_index()
    if return_type == 'Simple':
        returns = prices.pct_change()
    elif return_type == 'Log':
        returns = np.log(prices).diff()
    else:
        raise ValueError("Usa 'Simpe' o 'Log' para el Tipo de Cálculo de Retornos")
    return returns.dropna()

In [9]:
returns = calculate_returns(prices, return_type='Log')
returns.head()

Ticker,GOOG,JPM,LLY,NVDA,^GSPC
Date,,,,,
2020-01-03,-0.004919,-0.013284,-0.003334,-0.016135,-0.007085
2020-01-06,0.024358,-0.000795,0.003712,0.004185,0.003527
2020-01-07,-0.000624,-0.017147,0.001889,0.012034,-0.002807
2020-01-08,0.007849,0.007771,0.009015,0.001874,0.004890
2020-01-09,0.010984,0.003645,0.016393,0.010923,0.006633


In [10]:
rf = 0.03 / 252  # Daily risk-free rate assuming 3% annual

premium = returns - rf

def capm_regression(premium, market_returns):
    import statsmodels.api as sm

    results = {}
    
    for ticker in premium.columns:
        Y = premium[ticker]
        X = sm.add_constant(market_returns)
        model = sm.OLS(Y, X).fit()

        results[ticker] = {
            "Alpha": model.params["const"],
            "Beta": model.params[market_returns.name],
            "t-Alpha": model.tvalues["const"],
            "t-Beta": model.tvalues[market_returns.name],
            "R2": model.rsquared,
            "N": int(model.nobs)
        }

    return pd.DataFrame(results).T

In [11]:
RESULTS = capm_regression(returns.drop(columns="^GSPC"), returns["^GSPC"])
RESULTS["Expected Returns"] = rf * 252 + RESULTS["Beta"] * (returns["^GSPC"].mean() * 252)
RESULTS

,Alpha,Beta,t-Alpha,t-Beta,R2,N,Expected Returns
GOOG,0.000463,1.115851,1.275672,40.751257,0.526416,1496.0,0.168770
JPM,0.000130,1.069064,0.365142,39.685109,0.513182,1496.0,0.162952
LLY,0.001120,0.651155,2.223040,17.146556,0.164432,1496.0,0.110979
NVDA,0.001385,1.777856,2.261503,38.492536,0.497929,1496.0,0.251099
